In [1]:
# Import necessary libraries
import sqlite3
import pandas as pd

# Load datasets into Pandas DataFrames
marketing_cost_df = pd.read_csv("marketing_cost.csv")
reservations_df = pd.read_csv("reservations.csv")
merchants_df = pd.read_csv("merchants.csv")
marketing_mapping_df = pd.read_csv("marketing_mapping.csv")  # Mapping subchannel to channel

# Convert date columns to proper format
reservations_df["reservation_created_date"] = pd.to_datetime(reservations_df["reservation_created_date"])
marketing_cost_df["date"] = pd.to_datetime(marketing_cost_df["date"])

# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Load DataFrames into SQLite tables
marketing_cost_df.to_sql("marketing_cost", conn, index=False, if_exists="replace")
reservations_df.to_sql("reservations", conn, index=False, if_exists="replace")
merchants_df.to_sql("merchants", conn, index=False, if_exists="replace")
marketing_mapping_df.to_sql("marketing_mapping", conn, index=False, if_exists="replace")

4

In [3]:

# 1️⃣ Identify the Most Important Cities Based on Reservations
query_top_cities = """
SELECT 
    m.city, 
    m.merchant_country_iso3 AS country, 
    COUNT(r.reservation_id) AS total_reservations, 
    SUM(r.revenue) AS total_revenue
FROM reservations r
LEFT JOIN merchants m ON r.merchant_id = m.merchant_id
GROUP BY m.city, m.merchant_country_iso3
ORDER BY total_reservations DESC
LIMIT 10;
"""
top_cities = pd.read_sql(query_top_cities, conn)

# 2️⃣ Identify the Most Important Countries Based on Reservations
query_top_countries = """
SELECT 
    r.country, 
    COUNT(r.reservation_id) AS total_reservations, 
    SUM(r.revenue) AS total_revenue
FROM reservations r
GROUP BY r.country
ORDER BY total_reservations DESC
LIMIT 10;
"""
top_countries = pd.read_sql(query_top_countries, conn)

# 3️⃣ Aggregate Marketing Cost at the Marketing Channel Level
query_marketing_spend_per_channel = """
SELECT 
    mm.marketing_channel,
    SUM(mc.marketing_cost) AS total_marketing_spend
FROM marketing_cost mc
LEFT JOIN marketing_mapping mm
ON mc.marketing_subchannel = mm.marketing_subchannel
GROUP BY mm.marketing_channel;
"""
marketing_spend_per_channel = pd.read_sql(query_marketing_spend_per_channel, conn)

# 4️⃣ Calculate ROAS at the Marketing Channel Level
query_roas_per_channel = """
SELECT 
    r.marketing_channel,
    r.total_revenue,
    m.total_marketing_spend,
    COALESCE(r.total_revenue, 0) / NULLIF(m.total_marketing_spend, 0) AS ROAS
FROM 
    (SELECT 
        marketing_channel,
        SUM(revenue) AS total_revenue
    FROM reservations
    GROUP BY marketing_channel) r
LEFT JOIN
    (SELECT 
        mm.marketing_channel,
        SUM(mc.marketing_cost) AS total_marketing_spend
    FROM marketing_cost mc
    LEFT JOIN marketing_mapping mm
    ON mc.marketing_subchannel = mm.marketing_subchannel
    GROUP BY mm.marketing_channel) m
ON r.marketing_channel = m.marketing_channel
ORDER BY ROAS DESC;
"""

# Execute the query
roas_per_channel = pd.read_sql(query_roas_per_channel, conn)

# Close the database connection
conn.close()


In [4]:
# Display the results
print("Top Cities by Reservations & Revenue:")
display(top_cities)

print("Top Countries by Reservations & Revenue:")
display(top_countries)

print("ROAS by Marketing Channel:")
display(roas_per_channel)

Top Cities by Reservations & Revenue:


,city,country,total_reservations,total_revenue
0,Singapore,SG,391519,1.959500e+06
1,Wien,AUT,142646,7.130031e+05
2,Berlin,DEU,98547,4.930346e+05
3,Melbourne,AUS,81040,4.052284e+05
4,Köln,DEU,64393,3.203921e+05
5,Roma,IT,53342,2.667415e+05
6,München,DEU,47915,2.394149e+05
7,London,GB,46989,2.346579e+05
8,Sydney,AUS,37880,1.892935e+05
9,Helsinki,FIN,37761,1.886673e+05


Top Countries by Reservations & Revenue:


,country,total_reservations,total_revenue
0,DE,474200,2.369403e+06
1,SG,391519,1.959500e+06
2,AT,235220,1.174814e+06
3,IT,227285,1.135913e+06
4,AU,189071,9.452944e+05
5,GB,148228,7.405136e+05
6,FI,78305,3.921192e+05
7,TR,31249,1.556480e+05
8,NL,28897,1.448477e+05
9,CH,7318,3.627467e+04


ROAS by Marketing Channel:


,marketing_channel,total_revenue,total_marketing_spend,ROAS
0,Affiliate,4.288427e+05,1.418768e+04,30.226423
1,SEM,1.954881e+06,2.201503e+07,0.088798
2,Paid Social,1.442230e+00,1.152007e+03,0.001252
3,CRM,2.830407e+05,NaN,NaN
4,Direct,1.339345e+06,NaN,NaN
5,Organic Social,5.344237e+01,NaN,NaN
6,Other,2.762222e+05,NaN,NaN
7,Referral,4.278195e+06,NaN,NaN
8,SEO,5.294561e+05,NaN,NaN
